In [1]:
from keras.layers import Input, Dense
from keras.models import Model
from scipy.io import loadmat
import numpy as np
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

/home/hafez/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Input.mat --- low resolution PAN --- the patches created by the spationally degraded PAN image
# Target.mat --- high resolution PAN --- the patches created by the original PAN image
# generated from Data_Generation.m

lrPAN = loadmat('Input.mat')
lrPAN_input = lrPAN["Input"]
hrPAN = loadmat('Target.mat')
hrPAN_input = hrPAN["Target"]
[patch_size, number_of_patches] = lrPAN_input.shape

In [3]:
hrPAN_mat = np.zeros([number_of_patches, patch_size]) 
for i in range(number_of_patches):
    hrPAN_mat[i,:] = hrPAN_input[:,i]

In [4]:
lrPAN_mat = np.zeros([number_of_patches, patch_size]) 
for i in range(number_of_patches):
    lrPAN_mat[i,:] = lrPAN_input[:,i]

In [5]:
model_input = np.zeros([number_of_patches,8,8,1])
for i in range(len(lrPAN_mat)):
    model_input[i] = np.reshape(lrPAN_mat[i],[8,8,1])

In [6]:
model_output = np.zeros([number_of_patches,8,8,1])
for i in range(len(hrPAN_mat)):
    model_output[i] = np.reshape(hrPAN_mat[i],[8,8,1])

In [7]:
input_img = Input(shape=(8, 8, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)


x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)


x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
#encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)


x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)


x = Conv2D(16, (3, 3), activation='relu', padding = 'same')(x)
#x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='linear', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.summary()
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 8, 8, 1)           0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 16)          160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 8)           1160      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 8)           0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 8)           584       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2, 2, 8)           584       
__________

In [8]:
autoencoder.fit(model_input, model_output,
                epochs=30,
                batch_size=256,
                shuffle=True)

Epoch 1/4
115600/115600 [==============================] - 10s 83us/step - loss: 615.0644
Epoch 2/4
115600/115600 [==============================] - 9s 82us/step - loss: 353.9246
Epoch 3/4
115600/115600 [==============================] - 9s 77us/step - loss: 316.9757
Epoch 4/4
115600/115600 [==============================] - 9s 77us/step - loss: 287.1069


In [9]:
# Test.mat --- low resolution MS patches
### generated from Data_Generation.m (4 channels of MS (Blue, Green, Red, and NIR))

lrms = loadmat('Test.mat')
lrms_test = lrms["Test"]
[patch_size_test, number_of_patches_test] = lrms_test.shape
Test_Data = np.zeros([number_of_patches_test,patch_size])
for i in range(number_of_patches_test):
    Test_Data[i,:] = lrms_test[:,i]
Test_Data_reshaped = np.zeros([number_of_patches_test,8,8,1])
for i in range(len(Test_Data)):
    Test_Data_reshaped[i] = np.reshape(Test_Data[i],[8,8,1])

In [11]:
decoded_imgs = autoencoder.predict(Test_Data_reshaped)

In [12]:
decoded_imgs_reshaped = np.zeros([patch_size,number_of_patches_test]) 
for i in range(len(decoded_imgs)):
    decoded_imgs_reshaped[:,i] = np.reshape(decoded_imgs[i],patch_size)

In [13]:
import scipy.io as sio
sio.savemat('Estimated_HighLRMS.mat',{"Decoded":decoded_imgs_reshaped})